# préparation des données

In [5]:
import pandas as pd

In [12]:
df = pd.read_csv("Velib_Mai_2022.csv")

MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type int64

In [9]:
df.head()

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning,ping_time
0,Benjamin Godard - Victor Hugo,16107,2,1,"[48.865983, 2.275725]",2022-05-01T00:33:45+02:00,3,31,35.0,OUI,OUI,Paris,OUI,01/05/2022 00:50:01
1,Charonne - Robert et Sonia Delauney,11104,9,4,"[48.85590755596891, 2.3925706744194035]",2022-05-01T00:39:22+02:00,13,5,20.0,OUI,OUI,Paris,OUI,01/05/2022 00:50:01
2,Mairie du 12ème,12109,0,0,"[48.840855311763, 2.3875549435616]",2022-05-01T00:40:15+02:00,0,30,30.0,NON,OUI,Paris,NON,01/05/2022 00:50:01
3,Toudouze - Clauzel,9020,1,1,"[48.87929591733507, 2.3373600840568547]",2022-05-01T00:38:42+02:00,2,19,21.0,OUI,OUI,Paris,OUI,01/05/2022 00:50:01
4,Harpe - Saint-Germain,5001,2,34,"[48.85151881501689, 2.343670316040516]",2022-05-01T00:40:52+02:00,36,8,45.0,OUI,OUI,Paris,OUI,01/05/2022 00:50:01


* ebike : nombre de vélo électrique disponible
* mechanical : nombre de vélo mécanique disponible
* numbikesavailable : nombre de vélo disponible en total
* numdocksavailable : nombre de bornes dispo en total
* capacity : nombre de bornes en total
* is_renting: si la station prête de vélo
* is_installed : si la station est en opération
* is_returned : si la station accepte de vélo

# supprimer la colonne "ping" pour vérifier les doublons
# taux de duplication : 47%

In [10]:
df = df.iloc[:,0:-1]

In [11]:
df.duplicated().sum()

0

In [6]:
df.duplicated().sum()/df.shape[0]

0.4683941749470682

In [7]:
df = df.drop_duplicates(keep='first')

In [8]:
df.duplicated().sum()

0

# supprimer les lignes nulles (aucune information)

In [9]:
df.isnull().sum()

name                           4
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                4
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       4
is_renting                     0
is_installed                   0
nom_arrondissement_communes    4
is_returning                   0
dtype: int64

In [10]:
df.loc[df['name'].isna()]

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
1280662,NaN,13044_Relais,0,0,NaN,2022-05-07T09:59:00+02:00,0,0,NaN,OUI,OUI,NaN,OUI
2502169,NaN,45005,0,0,NaN,2022-05-13T16:00:22+02:00,0,0,NaN,NON,NON,NaN,NON
3600407,NaN,13118_Relais,0,0,NaN,2022-05-19T08:43:47+02:00,0,0,NaN,OUI,OUI,NaN,OUI
6061359,NaN,30005,0,0,NaN,2022-05-31T09:26:59+02:00,0,0,NaN,OUI,OUI,NaN,OUI


In [11]:
df = df.loc[df['name'].notna()]

# vérifier si toujours ebike+mechanical == numbikesavailable
# OUI 

In [18]:
condition = (df["ebike"] + df["mechanical"] != df["numbikesavailable"])

In [19]:
df.loc[condition]

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning


# verifier colonne stationcode : '31024_relais', '13118_Relais'
=> toutes les informations contient "relais" sont des données nulles - 794 lignes => supprimer

In [20]:
df['stationcode'].str.len().unique()

array([ 5,  4, 12], dtype=int64)

In [21]:
df_q = df.loc[df['stationcode'].str.len()==12]

In [22]:
df_q['stationcode'].unique()

array(['31024_relais', '13118_Relais'], dtype=object)

In [23]:
df.loc[(df['stationcode']=='31024') | (df['stationcode']=='13118')]

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
29,Romainville - Vaillant-Couturier,31024,5,1,"[48.86785, 2.446748]",2022-05-01T00:38:29+02:00,6,32,38.0,OUI,OUI,Montreuil,OUI
73,Regnault - Patay,13118,3,36,"[48.82435782216384, 2.377065159380436]",2022-05-01T00:40:52+02:00,39,20,63.0,OUI,OUI,Paris,OUI
2903,Regnault - Patay,13118,4,36,"[48.82435782216384, 2.377065159380436]",2022-05-01T00:40:52+02:00,40,19,63.0,OUI,OUI,Paris,OUI
5683,Romainville - Vaillant-Couturier,31024,5,2,"[48.86785, 2.446748]",2022-05-01T00:38:29+02:00,7,31,38.0,OUI,OUI,Montreuil,OUI
7102,Romainville - Vaillant-Couturier,31024,6,2,"[48.86785, 2.446748]",2022-05-01T00:38:29+02:00,8,30,38.0,OUI,OUI,Montreuil,OUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170736,Regnault - Patay,13118,4,38,"[48.82435782216384, 2.377065159380436]",2022-05-31T23:26:03+02:00,42,20,63.0,OUI,OUI,Paris,OUI
6173561,Romainville - Vaillant-Couturier,31024,8,1,"[48.86785, 2.446748]",2022-05-31T23:24:17+02:00,9,29,38.0,OUI,OUI,Montreuil,OUI
6174997,Regnault - Patay,13118,5,39,"[48.82435782216384, 2.377065159380436]",2022-05-31T23:26:03+02:00,44,18,63.0,OUI,OUI,Paris,OUI
6176403,Romainville - Vaillant-Couturier,31024,9,1,"[48.86785, 2.446748]",2022-05-31T23:24:17+02:00,10,28,38.0,OUI,OUI,Montreuil,OUI


In [24]:
df_q

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
93,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-01T00:41:04+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
8573,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-01T01:35:23+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
17060,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-01T02:35:32+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
25538,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-01T03:37:05+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
34021,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-01T04:39:54+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6136737,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-31T19:25:05+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
6145251,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-31T20:24:23+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
6153759,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-31T21:26:28+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI
6162284,Romainville - Vaillant-couturier,31024_relais,0,0,"[48.868054, 2.4470495]",2022-05-31T22:20:47+02:00,0,0,0.0,OUI,OUI,Montreuil,OUI


In [25]:
df = df.loc[df['stationcode'].str.len()!=12]

In [26]:
df['stationcode'].str.len().unique()

array([5, 4], dtype=int64)

# vérifier capacity == 0 : 2 lignes
=> 45005 non install 
=> 30005 installed le 31/05 : solution modifier : is_renting=non & is_returning=non

In [27]:
df.loc[df['capacity']==0]

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
2506444,Stalingrad - Général de Gaulle,45005,0,0,"[48.823450589614, 2.4965062363218]",2022-05-13T16:00:22+02:00,0,0,0.0,NON,NON,Champigny-sur-Marne,NON
6067022,Convention - Campus de Bobigny,30005,0,0,"[48.914920058264, 2.4172238067445]",2022-05-31T09:26:59+02:00,0,0,0.0,OUI,OUI,Bobigny,OUI


In [29]:
df.loc[df['stationcode']=='30005', 'is_renting'] = "NON"
df.loc[df['stationcode']=='30005', 'is_returning'] = "NON"
df.loc[df['stationcode']=='30005', 'is_installed'] = "NON"

# vérifier capacity < numdocksavailable : 2 lignes
=> 41203 capacity de 20 à 51 le 18/05 à 9H35 (installation de bornes) => solution modifier le capacity = 51

In [30]:
c = df['capacity'] - df['numdocksavailable']

In [31]:
q = df.loc[c<0]

In [32]:
q['stationcode'].unique()

array(['41203'], dtype=object)

In [33]:
q.loc[q['stationcode'] == '41203']

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
3410736,Stalingrad - Rigollots,41203,0,0,"[48.84997403844251, 2.45557576417923]",2022-05-18T09:35:36+02:00,0,50,20.0,OUI,OUI,Fontenay-sous-Bois,OUI
3412196,Stalingrad - Rigollots,41203,0,1,"[48.84997403844251, 2.45557576417923]",2022-05-18T09:35:36+02:00,1,50,20.0,OUI,OUI,Fontenay-sous-Bois,OUI
3413619,Stalingrad - Rigollots,41203,0,0,"[48.84997403844251, 2.45557576417923]",2022-05-18T09:35:36+02:00,0,51,20.0,OUI,OUI,Fontenay-sous-Bois,OUI
3416394,Stalingrad - Rigollots,41203,1,0,"[48.84997403844251, 2.45557576417923]",2022-05-18T10:35:10+02:00,1,50,20.0,OUI,OUI,Fontenay-sous-Bois,OUI


In [34]:
df.loc[c<0, 'capacity'] = 51.0

# vérifier numdocksavailable + numbikesavailable > capacity : 16401 lignes
* 23 stationcode : 
       '12105', '42004', '4005', '21302', '15056', '4104', '8004',
       '31023', '13123', '4109', '1006', '32012', '9104', '8002', '12020',
       '6024', '15070', '1119', '41203', '8117', '12009', '42010', '7021'
* solution : sauvegarder les données de ces stationcode dans un fichier indépendant

In [148]:
c = df['capacity'] < (df['numbikesavailable'] + df['numdocksavailable'])

In [149]:
q = df.loc[c]
q.shape

(0, 13)

In [87]:
print(q['stationcode'].unique())
print(q['stationcode'].nunique())

['12105' '42004' '4005' '21302' '15056' '4104' '8004' '31023' '13123'
 '4109' '1006' '32012' '9104' '8002' '12020' '6024' '15070' '1119' '41203'
 '8117' '12009' '42010' '7021']
23


In [88]:
list_stations_pb = q['stationcode'].unique()

In [89]:
c = df['stationcode'].isin(list_stations_pb)
df_pb_capcacity = df.loc[c]
df_pb_capcacity

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
199,Cossonnerie - Sébastopol,1119,1,4,"[48.86132665908402, 2.349419631063938]",2022-05-01T00:40:42+02:00,5,17,24.0,OUI,OUI,Paris,OUI
248,Mairie d'Ivry-sur-Seine,42010,9,41,"[48.81057120969284, 2.3842826485633855]",2022-05-01T00:40:13+02:00,50,2,53.0,OUI,OUI,Ivry-sur-Seine,OUI
274,Charenton - Diderot,12009,5,4,"[48.846171461187, 2.3793068155646]",2022-05-01T00:39:06+02:00,9,9,18.0,OUI,OUI,Paris,OUI
495,Balard - Montagne de la Fage,15070,5,28,"[48.83850740333351, 2.2782871127128606]",2022-05-01T00:41:00+02:00,33,2,36.0,OUI,OUI,Paris,OUI
587,Place François Mitterand,31023,3,0,"[48.86492782016328, 2.442958317697048]",2022-05-01T00:39:52+02:00,3,31,34.0,OUI,OUI,Montreuil,OUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6178058,Charenton - Diderot,12009,5,10,"[48.846171461187, 2.3793068155646]",2022-05-31T23:26:44+02:00,15,1,18.0,OUI,OUI,Paris,OUI
6178417,Mornay - Sully,4109,2,6,"[48.84970399548416, 2.364460788667202]",2022-05-31T23:26:44+02:00,8,6,15.0,OUI,OUI,Paris,OUI
6179174,Bercy - Villot,12105,1,5,"[48.841795009027, 2.3767850184816]",2022-05-31T23:26:13+02:00,6,25,34.0,OUI,OUI,Paris,OUI
6179180,Caumartin - Provence,9104,1,0,"[48.874422773426545, 2.3284685611724854]",2022-05-31T23:24:16+02:00,1,20,22.0,OUI,OUI,Paris,OUI


In [90]:
df_pb_capcacity['stationcode'].unique()
df_pb_capcacity['stationcode'].nunique()
df_pb_capcacity.to_csv('output1/pb_cap')

23

In [92]:
c = ~df['stationcode'].isin(list_stations_pb)
df_nopb = df.loc[c]
df_nopb['stationcode'].nunique()

1397

In [93]:
df = df_nopb

In [94]:
df['stationcode'].nunique()

1397

In [119]:
df.nunique()

name                             1396
stationcode                      1397
ebike                              60
mechanical                         72
coordonnees_geo                  1431
duedate                        321878
numbikesavailable                  74
numdocksavailable                  69
capacity                           63
is_renting                          2
is_installed                        2
nom_arrondissement_communes        61
is_returning                        2
dtype: int64

# doublons de coordonee_geo : prendre les coordonnées les plus précises (plus de longueur de string)

In [249]:
df_s = df.loc[:, ['name', 'stationcode', 'coordonnees_geo',  'nom_arrondissement_communes']]
df_s.shape

(3221977, 4)

In [250]:
df_s = df_s.drop_duplicates()
df_s.shape

(1432, 4)

In [251]:
df_s.shape[0] - df_s['stationcode'].nunique()

35

il y 35 lignes dupliquées en colonne coordonee_geo

In [252]:
t = df_s.loc[df_s.duplicated(subset='name', keep=False)].sort_values(by="name").reset_index()
list_station = t['stationcode'].unique()
t

,index,name,stationcode,coordonnees_geo,nom_arrondissement_communes
0,37074,Alexandre Dumas - Voltaire,11102,"[48.852888670365, 2.3891476408096]",Paris
1,335,Alexandre Dumas - Voltaire,11102,"[48.85288867036487, 2.389147640809553]",Paris
2,1416577,Arsène Houssaye - Champs-Elysées,8028,"[48.873409276267, 2.2977904602885]",Paris
3,1047,Arsène Houssaye - Champs-Elysées,8028,"[48.87340927626732, 2.2977904602885246]",Paris
4,1408256,Balzac - Champs Elysées,8052,"[48.872699639621, 2.3001953959465]",Paris
...,...,...,...,...,...
67,2600223,Square d'Anvers,9005,"[48.882633484311, 2.3448844062122]",Paris
68,451,Sèvres-Gallieni,21032,"[48.83780940895712, 2.2281519323587418]",Boulogne-Billancourt
69,2770312,Sèvres-Gallieni,21032,"[48.837809408957, 2.2281519323587]",Boulogne-Billancourt
70,1044,Traktir - Foch,16103,"[48.87275533461363, 2.2915497802487077]",Paris


In [253]:
# prendre les coordonnées en longueurs 17 et 18 
# comment on peut prendre le string le plus long ?
t['coordonnees_geo_info_str'] = t['coordonnees_geo'].str.len()
t

,index,name,stationcode,coordonnees_geo,nom_arrondissement_communes,coordonnees_geo_info_str
0,37074,Alexandre Dumas - Voltaire,11102,"[48.852888670365, 2.3891476408096]",Paris,34
1,335,Alexandre Dumas - Voltaire,11102,"[48.85288867036487, 2.389147640809553]",Paris,38
2,1416577,Arsène Houssaye - Champs-Elysées,8028,"[48.873409276267, 2.2977904602885]",Paris,34
3,1047,Arsène Houssaye - Champs-Elysées,8028,"[48.87340927626732, 2.2977904602885246]",Paris,39
4,1408256,Balzac - Champs Elysées,8052,"[48.872699639621, 2.3001953959465]",Paris,34
...,...,...,...,...,...,...
67,2600223,Square d'Anvers,9005,"[48.882633484311, 2.3448844062122]",Paris,34
68,451,Sèvres-Gallieni,21032,"[48.83780940895712, 2.2281519323587418]",Boulogne-Billancourt,39
69,2770312,Sèvres-Gallieni,21032,"[48.837809408957, 2.2281519323587]",Boulogne-Billancourt,34
70,1044,Traktir - Foch,16103,"[48.87275533461363, 2.2915497802487077]",Paris,39


In [254]:
t['str_diff'] = t['coordonnees_geo_info_str'].diff()
t

,index,name,stationcode,coordonnees_geo,nom_arrondissement_communes,coordonnees_geo_info_str,str_diff
0,37074,Alexandre Dumas - Voltaire,11102,"[48.852888670365, 2.3891476408096]",Paris,34,NaN
1,335,Alexandre Dumas - Voltaire,11102,"[48.85288867036487, 2.389147640809553]",Paris,38,4.0
2,1416577,Arsène Houssaye - Champs-Elysées,8028,"[48.873409276267, 2.2977904602885]",Paris,34,-4.0
3,1047,Arsène Houssaye - Champs-Elysées,8028,"[48.87340927626732, 2.2977904602885246]",Paris,39,5.0
4,1408256,Balzac - Champs Elysées,8052,"[48.872699639621, 2.3001953959465]",Paris,34,-5.0
...,...,...,...,...,...,...,...
67,2600223,Square d'Anvers,9005,"[48.882633484311, 2.3448844062122]",Paris,34,-5.0
68,451,Sèvres-Gallieni,21032,"[48.83780940895712, 2.2281519323587418]",Boulogne-Billancourt,39,5.0
69,2770312,Sèvres-Gallieni,21032,"[48.837809408957, 2.2281519323587]",Boulogne-Billancourt,34,-5.0
70,1044,Traktir - Foch,16103,"[48.87275533461363, 2.2915497802487077]",Paris,39,5.0


In [255]:
t = t.loc[t['str_diff']>0]

In [256]:
t.shape

(36, 7)

In [257]:
t = t.drop(columns=['coordonnees_geo_info_str', 'str_diff', 'index'])

In [258]:
t.columns

Index(['name', 'stationcode', 'coordonnees_geo',
       'nom_arrondissement_communes'],
      dtype='object')

In [259]:
df_s.index

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            4137646, 4798809, 4842482, 5368884, 5377777, 5377912, 5830895,
            5882172, 6049388, 6067022],
           dtype='int64', length=1432)

In [260]:
t1 = df_s.loc[~df_s['stationcode'].isin(list_station)]
t1.shape
t1.columns

Index(['name', 'stationcode', 'coordonnees_geo',
       'nom_arrondissement_communes'],
      dtype='object')

In [261]:
df_s = pd.concat([t1,t])
df_s

,name,stationcode,coordonnees_geo,nom_arrondissement_communes
0,Benjamin Godard - Victor Hugo,16107,"[48.865983, 2.275725]",Paris
1,Charonne - Robert et Sonia Delauney,11104,"[48.85590755596891, 2.3925706744194035]",Paris
2,Mairie du 12ème,12109,"[48.840855311763, 2.3875549435616]",Paris
3,Toudouze - Clauzel,9020,"[48.87929591733507, 2.3373600840568547]",Paris
4,Harpe - Saint-Germain,5001,"[48.85151881501689, 2.343670316040516]",Paris
...,...,...,...,...
63,Sarrette - Général Leclerc,14019,"[48.82503085292528, 2.326412869398186]",Paris
64,Square Alban Satragne,10018,"[48.875388227727306, 2.355956733335916]",Paris
66,Square d'Anvers,9005,"[48.882633484310816, 2.344884406212207]",Paris
68,Sèvres-Gallieni,21032,"[48.83780940895712, 2.2281519323587418]",Boulogne-Billancourt


In [262]:
t = df_s.loc[df_s.duplicated(subset='name', keep=False)].sort_values(by="name").reset_index()
t

,index,name,stationcode,coordonnees_geo,nom_arrondissement_communes
0,26,Custine - Ramey,18015,"[48.888735896947, 2.3469526693225]",Paris
1,27,Custine - Ramey,18015,"[48.888735896947054, 2.3469526693224907]",Paris
2,56,Pyrénées - Cours de Vincennes,20003,"[48.847796779935, 2.4061721563339]",Paris
3,57,Pyrénées - Cours de Vincennes,20003,"[48.84779677993506, 2.406172156333924]",Paris


In [263]:
df_s = df_s.drop_duplicates(subset='stationcode', keep='last')
df_s

,name,stationcode,coordonnees_geo,nom_arrondissement_communes
0,Benjamin Godard - Victor Hugo,16107,"[48.865983, 2.275725]",Paris
1,Charonne - Robert et Sonia Delauney,11104,"[48.85590755596891, 2.3925706744194035]",Paris
2,Mairie du 12ème,12109,"[48.840855311763, 2.3875549435616]",Paris
3,Toudouze - Clauzel,9020,"[48.87929591733507, 2.3373600840568547]",Paris
4,Harpe - Saint-Germain,5001,"[48.85151881501689, 2.343670316040516]",Paris
...,...,...,...,...
63,Sarrette - Général Leclerc,14019,"[48.82503085292528, 2.326412869398186]",Paris
64,Square Alban Satragne,10018,"[48.875388227727306, 2.355956733335916]",Paris
66,Square d'Anvers,9005,"[48.882633484310816, 2.344884406212207]",Paris
68,Sèvres-Gallieni,21032,"[48.83780940895712, 2.2281519323587418]",Boulogne-Billancourt


In [264]:
t = df_s.loc[df_s.duplicated(subset='name', keep=False)].sort_values(by="name").reset_index()
t

,index,name,stationcode,coordonnees_geo,nom_arrondissement_communes


# vérifier si la colonne is_installed change
trois stations passent en ouvert (de NON à OUI)
* 23204 ouvert le 07/05
* 4006 ouvert le 31/05
* 51008 ouvert le 23/05

In [265]:
is_install = df.groupby('stationcode')['is_installed'].unique().reset_index()
is_install['is_installed'].str.len().unique()

array([1, 2], dtype=int64)

In [266]:
is_install.loc[is_install['is_installed'].str.len() == 2]

,stationcode,is_installed
900,23204,"[NON, OUI]"
1058,4006,"[NON, OUI]"
1223,51008,"[NON, OUI]"


In [267]:
df.loc[(df['stationcode']=='23204') & (df['is_installed']=='OUI')]

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
1264063,Roland Garros - Jean Bouin,23204,0,0,"[48.809460857173, 2.2853198638301]",2022-05-07T08:01:50+02:00,0,28,28.0,NON,OUI,Châtillon,NON
1266923,Roland Garros - Jean Bouin,23204,0,0,"[48.809460857173, 2.2853198638301]",2022-05-07T08:02:25+02:00,0,28,28.0,NON,OUI,Châtillon,NON
1271180,Roland Garros - Jean Bouin,23204,0,1,"[48.809460857173, 2.2853198638301]",2022-05-07T08:01:50+02:00,1,27,28.0,NON,OUI,Châtillon,NON
1272595,Roland Garros - Jean Bouin,23204,0,1,"[48.809460857173, 2.2853198638301]",2022-05-07T09:02:32+02:00,1,27,28.0,OUI,OUI,Châtillon,OUI
1276839,Roland Garros - Jean Bouin,23204,0,1,"[48.809460857173, 2.2853198638301]",2022-05-07T09:01:57+02:00,1,27,28.0,OUI,OUI,Châtillon,OUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6146529,Roland Garros - Jean Bouin,23204,4,0,"[48.809460857173, 2.2853198638301]",2022-05-31T20:17:20+02:00,4,24,28.0,OUI,OUI,Châtillon,OUI
6153656,Roland Garros - Jean Bouin,23204,5,0,"[48.809460857173, 2.2853198638301]",2022-05-31T20:17:20+02:00,5,23,28.0,OUI,OUI,Châtillon,OUI
6155070,Roland Garros - Jean Bouin,23204,5,0,"[48.809460857173, 2.2853198638301]",2022-05-31T21:19:32+02:00,5,23,28.0,OUI,OUI,Châtillon,OUI
6163587,Roland Garros - Jean Bouin,23204,5,0,"[48.809460857173, 2.2853198638301]",2022-05-31T22:21:43+02:00,5,23,28.0,OUI,OUI,Châtillon,OUI


In [268]:
df.loc[(df['stationcode']=='4006') & (df['is_installed']=='OUI')]

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
6035460,Bourdon - Pont Morland,4006,1,0,"[48.8478103, 2.3659356]",2022-05-31T07:23:27+02:00,1,51,52.0,OUI,OUI,Paris,OUI
6036870,Bourdon - Pont Morland,4006,0,0,"[48.8478103, 2.3659356]",2022-05-31T07:23:27+02:00,0,52,52.0,OUI,OUI,Paris,OUI
6043934,Bourdon - Pont Morland,4006,1,0,"[48.8478103, 2.3659356]",2022-05-31T08:23:48+02:00,1,51,52.0,OUI,OUI,Paris,OUI
6046827,Bourdon - Pont Morland,4006,0,0,"[48.8478103, 2.3659356]",2022-05-31T08:23:48+02:00,0,52,52.0,OUI,OUI,Paris,OUI
6052487,Bourdon - Pont Morland,4006,0,0,"[48.8478103, 2.3659356]",2022-05-31T09:24:11+02:00,0,52,52.0,OUI,OUI,Paris,OUI
6055367,Bourdon - Pont Morland,4006,0,1,"[48.8478103, 2.3659356]",2022-05-31T09:24:11+02:00,1,51,52.0,OUI,OUI,Paris,OUI
6060983,Bourdon - Pont Morland,4006,0,1,"[48.8478103, 2.3659356]",2022-05-31T10:24:56+02:00,1,51,52.0,OUI,OUI,Paris,OUI
6069521,Bourdon - Pont Morland,4006,0,1,"[48.8478103, 2.3659356]",2022-05-31T11:25:00+02:00,1,51,52.0,OUI,OUI,Paris,OUI
6070927,Bourdon - Pont Morland,4006,0,2,"[48.8478103, 2.3659356]",2022-05-31T11:25:00+02:00,2,50,52.0,OUI,OUI,Paris,OUI
6076618,Bourdon - Pont Morland,4006,1,2,"[48.8478103, 2.3659356]",2022-05-31T11:25:00+02:00,3,49,52.0,OUI,OUI,Paris,OUI


In [269]:
df.loc[(df['stationcode']=='51008') & (df['is_installed']=='OUI')]

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning
4536442,Martyrs de Châteaubriant - Jean Grandel,51008,4,0,"[48.951432, 2.211214]",2022-05-23T22:51:58+02:00,4,34,38.0,OUI,OUI,Argenteuil,OUI
4544942,Martyrs de Châteaubriant - Jean Grandel,51008,4,0,"[48.951432, 2.211214]",2022-05-23T23:52:15+02:00,4,34,38.0,OUI,OUI,Argenteuil,OUI
4553419,Martyrs de Châteaubriant - Jean Grandel,51008,4,0,"[48.951432, 2.211214]",2022-05-24T00:52:39+02:00,4,34,38.0,OUI,OUI,Argenteuil,OUI
4561902,Martyrs de Châteaubriant - Jean Grandel,51008,4,0,"[48.951432, 2.211214]",2022-05-24T01:52:53+02:00,4,34,38.0,OUI,OUI,Argenteuil,OUI
4570394,Martyrs de Châteaubriant - Jean Grandel,51008,4,0,"[48.951432, 2.211214]",2022-05-24T02:53:09+02:00,4,34,38.0,OUI,OUI,Argenteuil,OUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6137852,Martyrs de Châteaubriant - Jean Grandel,51008,4,4,"[48.951432, 2.211214]",2022-05-31T19:20:04+02:00,8,30,38.0,OUI,OUI,Argenteuil,OUI
6146376,Martyrs de Châteaubriant - Jean Grandel,51008,4,4,"[48.951432, 2.211214]",2022-05-31T20:20:13+02:00,8,30,38.0,OUI,OUI,Argenteuil,OUI
6154863,Martyrs de Châteaubriant - Jean Grandel,51008,4,4,"[48.951432, 2.211214]",2022-05-31T21:20:22+02:00,8,30,38.0,OUI,OUI,Argenteuil,OUI
6163356,Martyrs de Châteaubriant - Jean Grandel,51008,4,4,"[48.951432, 2.211214]",2022-05-31T22:20:30+02:00,8,30,38.0,OUI,OUI,Argenteuil,OUI


# changer le type de données : coordonnées_geo

In [270]:
geo = df_s['coordonnees_geo'].str.replace('[', '')
geo = geo.str.replace(']', '')
geo = geo.str.split(pat=',',expand=True)
geo[0] = geo[0].astype('float')
geo[1] = geo[1].astype('float')
geo.reset_index()

C:\Users\xiaoa\AppData\Local\Temp/ipykernel_12340/4102966322.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  geo = df_s['coordonnees_geo'].str.replace('[', '')
C:\Users\xiaoa\AppData\Local\Temp/ipykernel_12340/4102966322.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  geo = geo.str.replace(']', '')


,index,0,1
0,0,48.865983,2.275725
1,1,48.855908,2.392571
2,2,48.840855,2.387555
3,3,48.879296,2.337360
4,4,48.851519,2.343670
...,...,...,...
1388,63,48.825031,2.326413
1389,64,48.875388,2.355957
1390,66,48.882633,2.344884
1391,68,48.837809,2.228152


In [271]:
df_s['latitude'] = geo[0]
df_s['longitude'] = geo[1]
df_s.head()

C:\Users\xiaoa\AppData\Local\Temp/ipykernel_12340/3699987433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['latitude'] = geo[0]
C:\Users\xiaoa\AppData\Local\Temp/ipykernel_12340/3699987433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['longitude'] = geo[1]


,name,stationcode,coordonnees_geo,nom_arrondissement_communes,latitude,longitude
0,Benjamin Godard - Victor Hugo,16107,"[48.865983, 2.275725]",Paris,48.865983,2.275725
1,Charonne - Robert et Sonia Delauney,11104,"[48.85590755596891, 2.3925706744194035]",Paris,48.855908,2.392571
2,Mairie du 12ème,12109,"[48.840855311763, 2.3875549435616]",Paris,48.840855,2.387555
3,Toudouze - Clauzel,9020,"[48.87929591733507, 2.3373600840568547]",Paris,48.879296,2.337360
4,Harpe - Saint-Germain,5001,"[48.85151881501689, 2.343670316040516]",Paris,48.851519,2.343670


In [272]:
df_s = df_s.drop(columns="coordonnees_geo")

# vérifier si la capacity change pour chaque station:
il y a deux stations :

* un qui installe 31 bornes en 18/05/2022 =>solution : on prend le calcul : (2017j + 5114j)/31j = 34 (capacity) pour l'analyse de suite
* un "2003" qui a capacity=33 dans une seule matinée le 9/05 => on suppose que c'est l'error, on prend sa capacity majorité 32 pour l'analyse de suite

In [273]:
capacity = df.groupby('stationcode')['capacity'].unique()
capacity = capacity.reset_index()

In [274]:
df_s =df_s.merge(capacity, on="stationcode", how="left")
df_s.loc[:,'capacity'] = df_s['capacity'].map(lambda x: x[0])


In [275]:
df_s

,name,stationcode,nom_arrondissement_communes,latitude,longitude,capacity
0,Benjamin Godard - Victor Hugo,16107,Paris,48.865983,2.275725,35.0
1,Charonne - Robert et Sonia Delauney,11104,Paris,48.855908,2.392571,20.0
2,Mairie du 12ème,12109,Paris,48.840855,2.387555,30.0
3,Toudouze - Clauzel,9020,Paris,48.879296,2.337360,21.0
4,Harpe - Saint-Germain,5001,Paris,48.851519,2.343670,45.0
...,...,...,...,...,...,...
1388,Sarrette - Général Leclerc,14019,Paris,48.825031,2.326413,22.0
1389,Square Alban Satragne,10018,Paris,48.875388,2.355957,30.0
1390,Square d'Anvers,9005,Paris,48.882633,2.344884,33.0
1391,Sèvres-Gallieni,21032,Boulogne-Billancourt,48.837809,2.228152,27.0


In [276]:
df_s.loc[df_s['stationcode']=='2003', 'capacity'] = 32.0

In [278]:
df_s['capacity'] = df_s['capacity'].astype('int64')

In [282]:
df['taux_borne_dispo'] = (df['numdocksavailable'] / df['capacity'])*100
df

,name,stationcode,ebike,mechanical,coordonnees_geo,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,nom_arrondissement_communes,is_returning,taux_borne_dispo
0,Benjamin Godard - Victor Hugo,16107,2,1,"[48.865983, 2.275725]",2022-05-01T00:33:45+02:00,3,31,35.0,OUI,OUI,Paris,OUI,88.571429
1,Charonne - Robert et Sonia Delauney,11104,9,4,"[48.85590755596891, 2.3925706744194035]",2022-05-01T00:39:22+02:00,13,5,20.0,OUI,OUI,Paris,OUI,25.000000
2,Mairie du 12ème,12109,0,0,"[48.840855311763, 2.3875549435616]",2022-05-01T00:40:15+02:00,0,30,30.0,NON,OUI,Paris,NON,100.000000
3,Toudouze - Clauzel,9020,1,1,"[48.87929591733507, 2.3373600840568547]",2022-05-01T00:38:42+02:00,2,19,21.0,OUI,OUI,Paris,OUI,90.476190
4,Harpe - Saint-Germain,5001,2,34,"[48.85151881501689, 2.343670316040516]",2022-05-01T00:40:52+02:00,36,8,45.0,OUI,OUI,Paris,OUI,17.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6179189,Châteaudun - Place de la Liberté,22507,3,3,"[48.90970830301211, 2.2454486480600626]",2022-05-31T23:26:37+02:00,6,17,26.0,OUI,OUI,La Garenne-Colombes,OUI,65.384615
6179193,Garibaldi - Gabriel Péri,34008,8,17,"[48.90574191139427, 2.331604420805889]",2022-05-31T23:26:24+02:00,25,3,30.0,OUI,OUI,Saint-Ouen-sur-Seine,OUI,10.000000
6179197,Ordener - Poissonniers,18023,24,9,"[48.891213890844654, 2.351288666219873]",2022-05-31T23:24:00+02:00,33,2,35.0,OUI,OUI,Paris,OUI,5.714286
6179207,Charonne - Alexandre Dumas,20014,1,4,"[48.85607741505607, 2.394869327545166]",2022-05-31T23:34:06+02:00,5,30,35.0,OUI,OUI,Paris,OUI,85.714286


In [286]:
moy_taux_borne = df.groupby('stationcode')['taux_borne_dispo'].mean().reset_index()
moy_taux_borne.sort_values(by="taux_borne_dispo",ascending=False)

,stationcode,taux_borne_dispo
31,10033,100.000000
1033,34012,100.000000
780,21112,100.000000
1058,4006,94.497041
823,21510,91.731990
...,...,...
1174,46001,11.516415
1165,44017,11.130144
1172,45004,7.265255
939,30005,NaN


In [290]:
df_s = df_s.merge(moy_taux_borne, how="left", on="stationcode")
df_s

,name,stationcode,nom_arrondissement_communes,latitude,longitude,capacity,taux_borne_dispo
0,Benjamin Godard - Victor Hugo,16107,Paris,48.865983,2.275725,35,86.480494
1,Charonne - Robert et Sonia Delauney,11104,Paris,48.855908,2.392571,20,67.440265
2,Mairie du 12ème,12109,Paris,48.840855,2.387555,30,52.260891
3,Toudouze - Clauzel,9020,Paris,48.879296,2.337360,21,79.253539
4,Harpe - Saint-Germain,5001,Paris,48.851519,2.343670,45,63.075601
...,...,...,...,...,...,...,...
1388,Sarrette - Général Leclerc,14019,Paris,48.825031,2.326413,22,73.793188
1389,Square Alban Satragne,10018,Paris,48.875388,2.355957,30,81.724590
1390,Square d'Anvers,9005,Paris,48.882633,2.344884,33,84.950515
1391,Sèvres-Gallieni,21032,Boulogne-Billancourt,48.837809,2.228152,27,48.926192


In [291]:
df_s.to_csv('output1/stations.csv')

# changer les types de données pour les colones duedate et capacity
# ajouter des colonnes :
* taux_borne_disponible (bornedisponible/capacity)*100
* hour (0-23H)
* day (1-31)
* dayofweek (monday - sunday)
* is_workday (1 oui, 0 non)

In [293]:
df = df.drop(columns=['name', 'coordonnees_geo', 'nom_arrondissement_communes'])
df

,stationcode,ebike,mechanical,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,is_returning,taux_borne_dispo
0,16107,2,1,2022-05-01T00:33:45+02:00,3,31,35.0,OUI,OUI,OUI,88.571429
1,11104,9,4,2022-05-01T00:39:22+02:00,13,5,20.0,OUI,OUI,OUI,25.000000
2,12109,0,0,2022-05-01T00:40:15+02:00,0,30,30.0,NON,OUI,NON,100.000000
3,9020,1,1,2022-05-01T00:38:42+02:00,2,19,21.0,OUI,OUI,OUI,90.476190
4,5001,2,34,2022-05-01T00:40:52+02:00,36,8,45.0,OUI,OUI,OUI,17.777778
...,...,...,...,...,...,...,...,...,...,...,...
6179189,22507,3,3,2022-05-31T23:26:37+02:00,6,17,26.0,OUI,OUI,OUI,65.384615
6179193,34008,8,17,2022-05-31T23:26:24+02:00,25,3,30.0,OUI,OUI,OUI,10.000000
6179197,18023,24,9,2022-05-31T23:24:00+02:00,33,2,35.0,OUI,OUI,OUI,5.714286
6179207,20014,1,4,2022-05-31T23:34:06+02:00,5,30,35.0,OUI,OUI,OUI,85.714286


In [294]:
df['capacity'] = df['capacity'].astype('int64')

In [295]:
df['duedate'].str.len().unique()

array([25], dtype=int64)

In [296]:
datetimestr = df['duedate'].str.split(pat='T',expand=True)
datetimestr.reset_index()
date = pd.to_datetime(datetimestr[0], format="%Y-%m-%d")
date

0         2022-05-01
1         2022-05-01
2         2022-05-01
3         2022-05-01
4         2022-05-01
             ...    
6179189   2022-05-31
6179193   2022-05-31
6179197   2022-05-31
6179207   2022-05-31
6179232   2022-05-31
Name: 0, Length: 3221977, dtype: datetime64[ns]

In [297]:
hour = datetimestr[1].str[0:2].astype('int64')
df['date'] = date
df['hour'] = hour
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3221977 entries, 0 to 6179232
Data columns (total 13 columns):
 #   Column             Dtype         
---  ------             -----         
 0   stationcode        object        
 1   ebike              int64         
 2   mechanical         int64         
 3   duedate            object        
 4   numbikesavailable  int64         
 5   numdocksavailable  int64         
 6   capacity           int64         
 7   is_renting         object        
 8   is_installed       object        
 9   is_returning       object        
 10  taux_borne_dispo   float64       
 11  date               datetime64[ns]
 12  hour               int64         
dtypes: datetime64[ns](1), float64(1), int64(6), object(5)
memory usage: 344.1+ MB


In [298]:
df['dayofweek'] = df['date'].dt.dayofweek
df['dayofweek'] = df['dayofweek'].replace({0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
df.tail()

,stationcode,ebike,mechanical,duedate,numbikesavailable,numdocksavailable,capacity,is_renting,is_installed,is_returning,taux_borne_dispo,date,hour,dayofweek
6179189,22507,3,3,2022-05-31T23:26:37+02:00,6,17,26,OUI,OUI,OUI,65.384615,2022-05-31,23,Tuesday
6179193,34008,8,17,2022-05-31T23:26:24+02:00,25,3,30,OUI,OUI,OUI,10.000000,2022-05-31,23,Tuesday
6179197,18023,24,9,2022-05-31T23:24:00+02:00,33,2,35,OUI,OUI,OUI,5.714286,2022-05-31,23,Tuesday
6179207,20014,1,4,2022-05-31T23:34:06+02:00,5,30,35,OUI,OUI,OUI,85.714286,2022-05-31,23,Tuesday
6179232,20116,6,3,2022-05-31T23:25:28+02:00,9,12,22,OUI,OUI,OUI,54.545455,2022-05-31,23,Tuesday


In [299]:
df['day'] = df['date'].dt.day

In [300]:
df['workday'] = 1
# si c'est weekend ou jour férié, worday =0
c = ((df['dayofweek']=="Saturday") | (df['dayofweek']=="Sunday") | (df['date'] =="2022-05-26"))
df.loc[c, 'workday'] = 0

In [302]:
df.to_csv('output1/stationstatus.csv')